# Programs with arguments and saved results

In this document, we will learn how to pass arguments to our program and save results.

Let's create another file with our program [./source_files/program_2.py](./source_files/program_2.py). 

Instead of having the circuit defined inside the program (like we did in first example), we will pass it as an argument. We will also save the results, so we can access them later by calling `save_result()`.

Here is the program:

```python
# source_files/program_2.py

from quantum_serverless import get_arguments, save_result

from qiskit.primitives import Sampler

# get all arguments passed to this program
arguments = get_arguments()

# get specific argument that we are interested in
circuit = arguments.get("circuit")

sampler = Sampler()

quasi_dists = sampler.run(circuit).result().quasi_dists

print(f"Quasi distribution: {quasi_dists[0]}")

# saving results of a program
save_result({
    "quasi_dists": quasi_dists[0]
})
```

As you can see, the circuit construction is not inside the program anymore. Instead, we parse the arguments by calling the `get_arguments()` function.

Another change is the call to the `save_result()` function, which saves the results of the execution as serialized JSON.

Let's create and configure out client

In [1]:
from quantum_serverless import QuantumServerless, GatewayProvider
import os

In [2]:
provider = GatewayProvider(
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

Next we will create circuit that we want to pass as an argument to the program.

In [20]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()
circuit.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1

And finally we are creating our program and running it using `run` method and passing `arguments` parameter in form of dictionary with `circuit` key and our circuit as value

In [21]:
from quantum_serverless import Program

program = Program(
    title="Program with arguments",
    entrypoint="program_2.py",
    working_dir="./source_files/"
)

job = serverless.run(program, arguments={"circuit": circuit})
job

<Job | 4cf769b2-97d4-4aab-ab0e-832be87dfc43>

In [24]:
job.result()

{'quasi_dists': {'0': 0.4999999999999999, '3': 0.4999999999999999}}